In [2]:
%pip install undetected-chromedriver

In [ ]:
# using selenium to open an undetected browser and get to https://chatgpt.com/, send a message and get the response

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import os
import pandas as pd

# read lines from Chinese_couplet_dataset_traditional_U30.txt
with open(r'D:\duilianji\duilianji\guwen-biaodian\Chinese_couplet_dataset_traditional_U30.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

if not os.path.exists('output.csv'):
    with open('output.csv', 'w', encoding='utf-8') as f:
        f.write('input\toutput\n')

startIndex = 0
if os.path.exists('output.csv'):
    df = pd.read_csv('output.csv', sep='\t', encoding='utf-8')
    startIndex = len(df) * 2

for i in range(startIndex, len(lines), 2):
    options = uc.ChromeOptions()
    options.headless = False
    driver = uc.Chrome(options=options)
    driver.get('https://chatgpt.com/')
    time.sleep(1)

    # send a message
    # find all textarea elements
    input = driver.find_element(By.XPATH, '//*[@id="prompt-textarea"]')
    prompt = '''Given a couplet in Chinese with its phonetic translation and meaning translation like this:
"萬金寶劍藏秋水
一片冰心在玉壺"
is translated into
{
"phonetic" : "Vạn kim bảo kiếm tàng thu thủy, 
Nhất phiến băng tâm tại ngọc hồ."
"translation": "Bảo kiếm ngàn vàng ẩn dưới nước thu,
Lòng băng một mảnh đựng nơi bầu ngọc."
}
"慶瑞一峰高,玉珮築鯨傳勝跡
釣臺雙廟峙,靈金耀斗護神光"
is translated into
{
"phonetic" : "Khánh Thụy nhất phong cao, Ngọc Bội trúc kình truyền thắng tích
Điếu Đài song miếu trĩ, linh kim diệu Đẩu hộ thần quang."
"translation": "Một cung Khánh Thụy vút cao, Ngọc Bội ghi công truyền thắng tích,
Hai miếu Điếu đài sừng sững, ánh vàng rọi Đẩu hộ thần quang."
}

Help me to translate this couplet, only output the final result:
"''' + lines[i] + lines[i + 1].strip("\n") + '\"'

    for part in prompt.split('\n'):
        input.send_keys(part)
        ActionChains(driver).key_down(Keys.SHIFT).key_down(Keys.ENTER).key_up(Keys.SHIFT).key_up(Keys.ENTER).perform()

    # find button with aria-label="Send prompt"
    send_button = driver.find_element(By.XPATH, '//button[@aria-label="Send prompt"]')
    send_button.click()

    time.sleep(1)

    while True:
        try:
            # find element with class result-streaming
            response = driver.find_elements(By.CLASS_NAME, 'result-streaming')
            if len(response) == 0:
                break
            time.sleep(0.5)
        except Exception as e:
            print(e)
            break

    # find element with data-message-author-role="assistant"
    response = driver.find_element(By.XPATH, '//div[@data-message-author-role="assistant"]').text
    df = pd.DataFrame({'input': [lines[i] + lines[i + 1]], 'output': [response]})
    df.to_csv('output.csv', mode='a', header=False, index=False, encoding='utf-8', sep='\t')

    driver.quit()